# Importamos librerias

In [1]:
#pip install snowflake-connector-python==2.7.6

In [2]:
import pandas as pd
import numpy as np
import snowflake.connector
import json
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
pd.options.mode.chained_assignment = None

In [3]:
while True:
    try:

        f = open('credentials.json')
        data_pass = json.load(f)

        pass_ = input("INGRESAR PASSCODE:")

        ctx = snowflake.connector.connect(
            user = data_pass['snow']['USER'],
            password = data_pass['snow']['PASS'],
            account = data_pass['snow']['ACCOUNT'],
            passcode = pass_,
            database = 'SANDBOX_PLUS',
            schema = 'DWH'
        )

        cursor = ctx.cursor()

        print('Connected')

        break

    except:
        print('Incorrect Password - provide again')

    print('Correct Password - connected to SNOWFLAKE')

Connected


In [223]:
# url = 'https://docs.google.com/spreadsheets/d/1ShQ5G-iKtpxHwcHAIP5aO-CGZsInpOLKZXKV_LzTDY8/export?format=csv'
# df_1 = pd.read_csv(url, dtype = {'ITEM':str})
# df_1.head(2)

,GROUP_NAME,DEPT_NAME,ITEM,ARTC_ARTC_DESC,DAR DE BAJA,TICKETS_CADA_1000,UNIDADES_TY,VTA_SIN_IVA_TY,NUMERO_LOCALES_ACTIVOS,PROMOS_DISTINTAS_3M
0,BEBIDAS,BEB. ALCOHOLICAS,1000048327,CERVEZA SCHNEIDER LATA 473.00 U,FALSE,1.790,17507.0,"$989,252.30",95,1
1,BEBIDAS,BEB. ALCOHOLICAS,1000048614,VINO FINO BLANCO RESERVADO CONCHA Y TORO SAUVI...,FALSE,0.145,556.0,"$112,688.70",4,0


# Clase & Subclase

In [245]:
query = '''
SELECT
    CLA.CLASS_NAME,
    SUB.SUB_NAME,
    IM.ITEM
FROM
    MSTRDB.DWH.ITEM_MASTER AS IM
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
WHERE
    IM.ITEM IN {articulos}
'''

cursor.execute(query.format(articulos = tuple(df_1['ITEM'])))
cla_sub = cursor.fetch_pandas_all()
cla_sub.head(2)

,CLASS_NAME,SUB_NAME,ITEM
0,TOMATADOS,CONCENTRADOS,1000051749
1,OTROS SNACKS,PALITOS,1000053404


In [225]:
# df_1 = df_1.merge(cla_sub, on = 'ITEM', how = 'inner')
# df_1.head(2)

,GROUP_NAME,DEPT_NAME,ITEM,ARTC_ARTC_DESC,DAR DE BAJA,TICKETS_CADA_1000,UNIDADES_TY,VTA_SIN_IVA_TY,NUMERO_LOCALES_ACTIVOS,PROMOS_DISTINTAS_3M,CLASS_NAME,SUB_NAME
0,BEBIDAS,BEB. ALCOHOLICAS,1000048327,CERVEZA SCHNEIDER LATA 473.00 U,FALSE,1.790,17507.0,"$989,252.30",95,1,CERVEZAS,CERVEZAS IMPORTADAS
1,BEBIDAS,BEB. ALCOHOLICAS,1000048614,VINO FINO BLANCO RESERVADO CONCHA Y TORO SAUVI...,FALSE,0.145,556.0,"$112,688.70",4,0,VINOS FINOS,BLANCO FINO CONSIGNACIÓN


# Fuente 1: TICKETS_CADA_1000 / UNIDADES_TY / VTA_SIN_IVA_TY

In [247]:
query = '''
SELECT
    ORIN,
    TICKETS_CADA_1000,
    UNIDADES_TY,
    VTA_SIN_IVA_TY
FROM
    SANDBOX_PLUS.DWH.MAESTRO_PRODUCTOS_HIST
WHERE
    MES = '2023-11-01'
'''

cursor.execute(query)
fuente_1 = cursor.fetch_pandas_all()
fuente_1.head(2)

,ORIN,TICKETS_CADA_1000,UNIDADES_TY,VTA_SIN_IVA_TY
0,1000008406,NaN,NaN,NaN
1,1000008406,NaN,NaN,NaN


In [248]:
# # fuente_2 = fuente_1.groupby(['ORIN'])['TICKETS_CADA_1000', 'UNIDADES_TY', 'VTA_SIN_IVA_TY'].agg({'TICKETS_CADA_1000':'median', 'UNIDADES_TY':'sum', 'VTA_SIN_IVA_TY':'sum'}).reset_index()
# fuente_2.head(2)

C:\Users\leonardo.mangold\AppData\Local\Temp\ipykernel_18764\2056627805.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  fuente_2 = fuente_1.groupby(['ORIN'])['TICKETS_CADA_1000', 'UNIDADES_TY', 'VTA_SIN_IVA_TY'].agg({'TICKETS_CADA_1000':'median', 'UNIDADES_TY':'sum', 'VTA_SIN_IVA_TY':'sum'}).reset_index()


,ORIN,TICKETS_CADA_1000,UNIDADES_TY,VTA_SIN_IVA_TY
0,1000002394,NaN,0.0,0.0
1,1000005515,0.0,0.0,0.0


In [251]:
# # df_2 = df_1.merge(fuente_2, left_on = 'ITEM', right_on = 'ORIN', how = 'left')
# print(df_2.shape[0])
# df_2.head(2)

305


,GROUP_NAME,DEPT_NAME,ITEM,ARTC_ARTC_DESC,DAR DE BAJA,TICKETS_CADA_1000_x,UNIDADES_TY_x,VTA_SIN_IVA_TY_x,NUMERO_LOCALES_ACTIVOS,PROMOS_DISTINTAS_3M,...,SUB_NAME,R_DEPT_NAME,R_CLASE,R_SUBCLASE,ORIN_x,CANASTA,ORIN_y,TICKETS_CADA_1000_y,UNIDADES_TY_y,VTA_SIN_IVA_TY_y
0,BEBIDAS,BEB. ALCOHOLICAS,1000048327,CERVEZA SCHNEIDER LATA 473.00 U,FALSE,1.790,17507.0,"$989,252.30",95,1,...,CERVEZAS IMPORTADAS,5.0,5.0,3.0,1000048327,referente,1000048327,1.790,17507.0,989252.26
1,BEBIDAS,BEB. ALCOHOLICAS,1000048614,VINO FINO BLANCO RESERVADO CONCHA Y TORO SAUVI...,FALSE,0.145,556.0,"$112,688.70",4,0,...,BLANCO FINO CONSIGNACIÓN,77.0,6.0,2.0,1000048614,referente,1000048614,0.145,556.0,112688.71


# Fuente 2: NUMERO_LOCALES_ACTIVOS

In [229]:
# # query = '''
# SELECT
#     LAA.ORIN,
#     COUNT(DISTINCT FS.GEOG_LOCL_ID) AS NUMERO_LOCALES_ACTIVOS
# FROM
#     MSTRDB.DWH.FT_STOCK AS FS
#     INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FS.ARTC_ARTC_ID
# WHERE
#     FS.TIEM_DIA_ID = CURRENT_DATE() - 1
#     AND FS.STCK_ARTC_ACTIVO = 1
#     AND ORIN IN {articulos}
# GROUP BY
#     ALL
# '''
#
# tuple_1 = tuple(df_1['ITEM'])
# new_tuple = tuple(str(item) for item in tuple_1)
#
# cursor.execute(query.format(articulos = new_tuple))
# fuente_2 = cursor.fetch_pandas_all()
# fuente_2.head(2)

In [230]:
# # df_3 = df_2.merge(fuente_2, on = 'ORIN', how = 'left')
# df_3['NUMERO_LOCALES_ACTIVOS'].fillna(0, inplace = True)
# print(df_3.shape[0])
# df_3.drop(['ORIN'], axis = 1, inplace = True)
# df_3.head(2)

# Fuente 3: PROMOS_DISTINTAS_3M

In [231]:
# # query = '''
# SELECT
#     LPTO.*,
#     LPE.EVENTO_DESC,
#     LPE.EVENTO_ID,
#     FP.PROM_FECHA_INICIO AS INICIO,
#     FP.PROM_FECHA_FIN AS FIN,
#     CLA.CLASE,
#     CLA.CLASS_NAME,
#     SUB.SUBCLASE,
#     SUB.SUB_NAME,
#     LAA.ORIN,
#     LAA.ARTC_ARTC_DESC,
#     FP.GEOG_LOCL_ID,
#     LGL.GEOG_LOCL_COD,
#     LGL.GEOG_LOCL_DESC
# FROM
#     MSTRDB.DWH.FT_PROMOS AS FP
#     INNER JOIN MSTRDB.DWH.LU_PROM_EVENTO AS LPE ON LPE.EVENTO_ID = FP.EVENTO_ID
#     INNER JOIN MSTRDB.DWH.LU_PROM_TIPO_OFERTA AS LPTO ON LPTO.TIPO_OFERTA_ID = LPE.TIPO_OFERTA_ID
#     INNER JOIN MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA ON LAA.ARTC_ARTC_ID = FP.ARTC_ARTC_ID
#     INNER JOIN MSTRDB.DWH.LU_GEOG_LOCAL AS LGL ON LGL.GEOG_LOCL_ID = FP.GEOG_LOCL_ID
#     INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON IM.ITEM = LAA.ORIN
#     INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
#     INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
# WHERE
#     LPTO.TIPO_OFERTA_ID IN (10, 11, 12)
#     AND (TO_VARCHAR(FP.PROM_FECHA_INICIO, 'YYYY-MM')) IN ('2023-09', '2023-10', '2023-11')
# '''
#
# cursor.execute(query)
# fuente_3 = cursor.fetch_pandas_all()
# fuente_3.head(2)

In [232]:
# # fuente_3_parte_2 = fuente_3.groupby('ORIN')['EVENTO_DESC'].nunique().reset_index()
# fuente_3_parte_2.rename({'EVENTO_DESC':'PROMOS_DISTINTAS_3M'}, axis = 1, inplace = True)
# df_4 = df_3.merge(fuente_3_parte_2, left_on = 'ITEM', right_on = 'ORIN', how = 'left')
# df_4['PROMOS_DISTINTAS_3M'].fillna(0, inplace = True)
# df_4.drop(['ORIN'], axis = 1, inplace = True)
# print(df_4.shape[0])
# df_4.head(2)

In [233]:
# # df_4.to_excel('s.xlsx', index = False)

# Fuente 4: Rankings

In [234]:
# query = '''
WITH SUBCLASES AS
    (
    SELECT
        DISTINCT SUBCLASE
    FROM
        MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA
        INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON LAA.ORIN = IM.ITEM
    WHERE
        LAA.ORIN IN {articulos}
    )

SELECT
    GRO.GROUP_NAME,
    DEPS.DEPT_NAME,
    IM.CLASE,
    IM.SUBCLASE,
    LAA.ARTC_ARTC_COD,
    LAA.ORIN,
    LAA.ARTC_ARTC_DESC AS PRODUCTO,
    SUM(FV.VNTA_IMPORTE_SIN_IVA) AS VENTAS
FROM
    MSTRDB.DWH.LU_ARTC_ARTICULO AS LAA
    INNER JOIN MSTRDB.DWH.ITEM_MASTER AS IM ON LAA.ORIN = IM.ITEM
    INNER JOIN MSTRDB.DWH.SUBCLASS AS SUB ON SUB.SUBCLASE = IM.SUBCLASE
    INNER JOIN MSTRDB.DWH.CLASS AS CLA ON CLA.CLASE = IM.CLASE
    INNER JOIN MSTRDB.DWH.DEPS AS DEPS ON DEPS.DEPT = IM.DEPT
    INNER JOIN MSTRDB.DWH.GROUPS AS GRO ON GRO.GROUP_NO = DEPS.GROUP_NO
    INNER JOIN MSTRDB.DWH.FT_VENTAS AS FV ON FV.ARTC_ARTC_ID = LAA.ARTC_ARTC_ID
WHERE
    IM.SUBCLASE IN (SELECT * FROM SUBCLASES)
    AND FV.TIEM_DIA_ID >= DATEADD(MONTH, -1, CURRENT_DATE())
GROUP BY
    ALL
'''

tuple_1 = tuple(df_1['ITEM'])
new_tuple = tuple(str(item) for item in tuple_1)

cursor.execute(query.format(articulos = new_tuple))
rankings = cursor.fetch_pandas_all()
rankings.head(2)

,GROUP_NAME,DEPT_NAME,CLASE,SUBCLASE,ARTC_ARTC_COD,ORIN,PRODUCTO,VENTAS
0,COMESTIBLES,ADEREZOS,120003,1200030003,251295000,1000178772,VINAGRE DE ALCOHOL TATA 1L. 1,385208.77
1,LACTEOS,YOGURES,790001,7900010002,230553000,1000348161,YOGUR YOGURISIMO NATURAL SACHET 1.2 LT. NATURAL,159497.58


## Ranking 1: DEPT_NAME

In [235]:
# rankings.sort_values(by=['DEPT_NAME', 'VENTAS'], ascending=[True, False], inplace=True)

rankings['R_DEPT_NAME'] = rankings.groupby('DEPT_NAME').cumcount() + 1

## Ranking 2: CLASE

In [236]:
# rankings.sort_values(by = ['CLASE', 'VENTAS'], ascending = [True, False], inplace = True)
rankings['R_CLASE'] = rankings.groupby('CLASE').cumcount() + 1

## Ranking 3: SUBCLASE

In [237]:
# rankings.sort_values(by = ['SUBCLASE', 'VENTAS'], ascending = [True, False], inplace = True)
rankings['R_SUBCLASE'] = rankings.groupby('SUBCLASE').cumcount() + 1

In [238]:
# df_1 = df_1.merge(rankings[['ORIN', 'R_DEPT_NAME', 'R_CLASE', 'R_SUBCLASE']], left_on = 'ITEM', right_on = 'ORIN', how = 'left')
df_1.drop(['ORIN'], axis = 1, inplace = True)

In [239]:
# # for i in ['R_DEPT_NAME', 'R_CLASE', 'R_SUBCLASE']:
#     df_1[i] = df_1[i].astype(int)

# Canasta

In [240]:
# query = '''
SELECT
    DISTINCT
    ORIN,
    CANASTA
FROM
    (
    SELECT
        *
    FROM
        SANDBOX_PLUS.DWH.MAESTRO_PRODUCTOS_HIST
    WHERE
        MES = (SELECT MAX(MES) FROM SANDBOX_PLUS.DWH.MAESTRO_PRODUCTOS_HIST)
    )
WHERE
    ORIN IN {articulos}
'''

cursor.execute(query.format(articulos = tuple(df_1['ITEM'])))
canasta = cursor.fetch_pandas_all()
canasta.head(2)

,ORIN,CANASTA
0,1000172664,referente
1,1000349462,referente


In [241]:
# a = canasta.groupby('ORIN')['CANASTA'].nunique().reset_index()
a[a['CANASTA'] > 1]

,ORIN,CANASTA


In [242]:
# df_1 = df_1.merge(canasta, left_on = 'ITEM', right_on = 'ORIN', how = 'left')
df_1.head(2)

,GROUP_NAME,DEPT_NAME,ITEM,ARTC_ARTC_DESC,DAR DE BAJA,TICKETS_CADA_1000,UNIDADES_TY,VTA_SIN_IVA_TY,NUMERO_LOCALES_ACTIVOS,PROMOS_DISTINTAS_3M,CLASS_NAME,SUB_NAME,R_DEPT_NAME,R_CLASE,R_SUBCLASE,ORIN,CANASTA
0,BEBIDAS,BEB. ALCOHOLICAS,1000048327,CERVEZA SCHNEIDER LATA 473.00 U,FALSE,1.790,17507.0,"$989,252.30",95,1,CERVEZAS,CERVEZAS IMPORTADAS,5.0,5.0,3.0,1000048327,referente
1,BEBIDAS,BEB. ALCOHOLICAS,1000048614,VINO FINO BLANCO RESERVADO CONCHA Y TORO SAUVI...,FALSE,0.145,556.0,"$112,688.70",4,0,VINOS FINOS,BLANCO FINO CONSIGNACIÓN,77.0,6.0,2.0,1000048614,referente


In [243]:
# df_1.to_excel('s.xlsx', index = False)

In [244]:
# df_1.shape

(305, 17)

# Fin